In [1]:
#  Install and import required packages

!pip install -q transformers sentence-transformers torch spacy nltk tqdm
!pip install -U -q google-generativeai

import nltk, spacy
nltk.download('punkt')
nltk.download('punkt_tab')
!python -m spacy download en_core_web_sm


# importing the required libraries
import os, re, glob, time, csv, math, random
import pandas as pd
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
import google.generativeai as genai


# Load spaCy English model (for later optional use)
nlp = spacy.load("en_core_web_sm")

print("....Packages installed and models loaded successfully!....")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
....Packages installed and models loaded successfully!....


In [ ]:
# helper functions

# Defining vague word list
VAGUE_LEXICON = [
    "fast", "quick", "efficient", "easy", "simple", "user-friendly", "user friendly", "intuitive",
    "robust", "reliable", "secure", "flexible", "scalable", "sufficient", "adequate",
    "as soon as possible", "soon", "minimize", "optimize", "better", "improve",
    "high performance", "low latency", "fastly"
]

# Precompiling regex patterns for all vague words 
vague_patterns = [re.compile(r"\b" + re.escape(x) + r"\b", flags=re.I) for x in VAGUE_LEXICON]

print(f"printing vague patterns: {vague_patterns}")


def clean_text(raw_text: str) -> str:
    """Cleans text: removes artifacts, newlines, and extra spaces."""
    text = raw_text.replace('\x00', ' ')                         # remove null chars
    text = re.sub(r'(\w)-\s*\n\s*(\w)', r'\1\2', text)          # fix hyphenation across lines
    text = re.sub(r'[\r\n]+', ' ', text)                        # remove newlines
    text = re.sub(r'\s+', ' ', text)                            # normalize spaces
    text = re.sub(r'Page\s*\d+\s*of\s*\d+', ' ', text, flags=re.I)
    text = re.sub(r'^\s*Confidential', '', text, flags=re.I)
    text = re.sub(r'[\u2022\u2023\-\*•◦]', ' ', text)           # remove bullets
    text = re.sub(r'\b\d+(\.\d+)+\b', ' ', text)                # remove section numbers like 3.2.1
    text = re.sub(r'\b\d+\.\b', ' ', text)
    return text.strip()


def is_valid_sentence(s: str) -> bool:
    """Filters out too-short or meaningless sentences."""
    s = s.strip()
    if len(s.split()) < 4:     # too short
        return False
    if len(s) > 600:           # too long (likely a paragraph)
        return False
    if not re.search(r'[a-zA-Z]', s):  # no text
        return False
    return True


def find_vague_terms(sentence: str):
    """Returns list of vague words found in a sentence."""
    found = []
    for pattern in vague_patterns:
        if pattern.search(sentence):
            found.append(pattern.pattern.strip("\\b").strip("\\b"))
    return list(set(found))  # unique only



printing vague patterns: [re.compile('\\bfast\\b', re.IGNORECASE), re.compile('\\bquick\\b', re.IGNORECASE), re.compile('\\befficient\\b', re.IGNORECASE), re.compile('\\beasy\\b', re.IGNORECASE), re.compile('\\bsimple\\b', re.IGNORECASE), re.compile('\\buser\\-friendly\\b', re.IGNORECASE), re.compile('\\buser\\ friendly\\b', re.IGNORECASE), re.compile('\\bintuitive\\b', re.IGNORECASE), re.compile('\\brobust\\b', re.IGNORECASE), re.compile('\\breliable\\b', re.IGNORECASE), re.compile('\\bsecure\\b', re.IGNORECASE), re.compile('\\bflexible\\b', re.IGNORECASE), re.compile('\\bscalable\\b', re.IGNORECASE), re.compile('\\bsufficient\\b', re.IGNORECASE), re.compile('\\badequate\\b', re.IGNORECASE), re.compile('\\bas\\ soon\\ as\\ possible\\b', re.IGNORECASE), re.compile('\\bsoon\\b', re.IGNORECASE), re.compile('\\bminimize\\b', re.IGNORECASE), re.compile('\\boptimize\\b', re.IGNORECASE), re.compile('\\bbetter\\b', re.IGNORECASE), re.compile('\\bimprove\\b', re.IGNORECASE), re.compile('\\bhig

In [ ]:
#  Preprocessing: Read, clean, and save text files
RAW_TXT_DIR = "/content/drive/MyDrive/NLP_ASG/Data/raw_txt"
all_files = [f for f in os.listdir(RAW_TXT_DIR) if f.endswith(".txt")]
print(f"Found {len(all_files)} text files.")

CLEANED_DIR = "/content/drive/MyDrive//NLP_ASG/Data/Cleaned"
os.makedirs(CLEANED_DIR, exist_ok=True)

for file in tqdm(all_files, desc="Cleaning files"):
    in_path = os.path.join(RAW_TXT_DIR, file)
    out_path = os.path.join(CLEANED_DIR, file)

    try:
        with open(in_path, "r", encoding="utf-8", errors="ignore") as f:
            raw_text = f.read()
        cleaned_text = clean_text(raw_text)
        with open(out_path, "w", encoding="utf-8") as f:
            f.write(cleaned_text)
    except Exception as e:
        print(f"Error processing {file}: {e}")

print("Cleaning complete!")
print(f"Cleaned files saved in: {CLEANED_DIR}")



Found 79 text files.


Cleaning files: 100%|██████████| 79/79 [01:06<00:00,  1.20it/s]

Cleaning complete!
Cleaned files saved in: /content/drive/MyDrive/Colab Notebooks/NLP_ASG/Data/Cleaned


In [ ]:
# configuring gemini ai for rewriting sentences
from google.colab import userdata

# set this environment inside collab key's 
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# make sure your key is set
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel("gemini-2.5-flash")



In [10]:

# Paths
CLEANED_DIR = "/content/drive/MyDrive/NLP_ASG/Data/Cleaned"
OUTPUT_PATH = "/content/drive/MyDrive/NLP_ASG/Outputs/Vague_Rewrites.csv"
columns = ["file", "vague_term(s)", "original_sentence", "suggested_rewrite"]

# Start fresh
if os.path.exists(OUTPUT_PATH):
    os.remove(OUTPUT_PATH)
pd.DataFrame(columns=columns).to_csv(OUTPUT_PATH, index=False, encoding="utf-8")

# Prompt template for full-file rewriting
PROMPT_TEMPLATE_FILE = """
You are an expert software requirements engineer.

Below are vague requirement sentences extracted from a software specification file.
Rewrite **each** sentence to be clear, specific, measurable, and testable.

Guidelines:
- Maintain IEEE-SRS tone ("The system shall...", "The software must...").
- Replace vague adjectives such as {all_vague_terms} with quantifiable or objective metrics.
- Return only the rewritten requirements, numbered 1., 2., 3.… (same count as input).
- Do **not** add commentary, introductions, or examples.

Vague requirement sentences:
{sentences}

Rewritten:
"""


cleaned_files = [f for f in os.listdir(CLEANED_DIR) if f.endswith(".txt")]
print(f"Found {len(cleaned_files)} cleaned files.")

for file in tqdm(cleaned_files, desc="Processing files"):
    file_path = os.path.join(CLEANED_DIR, file)
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    sentences = sent_tokenize(text)
    vague_data = []
    for s in sentences:
        if not is_valid_sentence(s):
            continue
        found_terms = [term for term in VAGUE_LEXICON if re.search(rf"\b{re.escape(term)}\b", s, flags=re.I)]
        if found_terms:
            vague_data.append((found_terms, s))

    print(f"File '{file}' — vague sentences: {len(vague_data)}")
    if not vague_data:
        continue

    all_vague_terms = sorted(set(sum([terms for terms, _ in vague_data], [])))
    joined_sentences = "\n".join([f"{i+1}. {s}" for i, (_, s) in enumerate(vague_data)])
    prompt = PROMPT_TEMPLATE_FILE.format(
        all_vague_terms=", ".join(all_vague_terms), sentences=joined_sentences
    )

    for attempt in range(3):
        try:
            print(f"Sending prompt for file: {file}")
            response = model.generate_content(prompt)
            rewritten_block = response.text.strip()
            print("Gemini response preview:\n", rewritten_block[:400])
            break
        except Exception as e:
            print(f"Rewrite failed for {file}: {e}")
            rewritten_block = f"[Rewrite failed: {e}]"
            time.sleep(2 + random.random())
    else:
        rewritten_block = "[Rewrite failed after retries]"

    rewritten_lines = [re.sub(r"^\d+\.\s*", "", ln).strip() for ln in rewritten_block.splitlines() if ln.strip()]
    vague_rows = []
    for i, (terms, s) in enumerate(vague_data):
        rewrite = rewritten_lines[i] if i < len(rewritten_lines) else "[No rewrite returned]"
        vague_rows.append([file, ", ".join(terms), s.strip(), rewrite])

    if vague_rows:
        pd.DataFrame(vague_rows, columns=columns).to_csv(
            OUTPUT_PATH, mode="a", header=False, index=False, encoding="utf-8"
        )
        print(f"Added {len(vague_rows)} rows for {file}")

    time.sleep(0.5)

print("File-wise detection & rewriting complete!")
print(f"Results saved to: {os.path.abspath(OUTPUT_PATH)}")




Found 79 cleaned files.


Processing files:   0%|          | 0/79 [00:00<?, ?it/s]

File '2004 - philips.txt' — vague sentences: 1
Sending prompt for file: 2004 - philips.txt
Gemini response preview:
 1. The system shall ensure all user-facing text is grammatically correct, free of spelling errors, avoids undefined technical jargon, and achieves a Flesch-Kincaid Grade Level score of 8 or lower.
Added 1 rows for 2004 - philips.txt


Processing files:   1%|▏         | 1/79 [00:07<10:23,  8.00s/it]

File '2007 - get real 0.2.txt' — vague sentences: 5
Sending prompt for file: 2007 - get real 0.2.txt
Gemini response preview:
 1. The Get Real website shall present content tailored for high school teens (ages 14-18) and enable users to locate any career profile or informational article within a maximum of three clicks from the homepage. Each career profile shall include a minimum of 500 words detailing educational requirements, daily tasks, and salary expectations, covering at least 20 distinct computer science career pa
Added 5 rows for 2007 - get real 0.2.txt


...

File '2001 - telescope.txt' — vague sentences: 11
Sending prompt for file: 2001 - telescope.txt
Gemini response preview:
 1. The system shall achieve a spatial resolution of 0.25 pixels through the utilization of a centroid algorithm.
2. The system shall implement an on/off control algorithm with a configurable hysteresis value, settable in increments of 0.1 units from 0.5 to 5.0 units.
3. The software shall transition to fast timing mode when one of the following conditions is met: (a) the spacecraft's pointing erro
Added 11 rows for 2001 - telescope.txt


Processing files:  99%|█████████▊| 78/79 [30:19<00:29, 29.20s/it]

File '2003 - tachonet.txt' — vague sentences: 8
Sending prompt for file: 2003 - tachonet.txt
Gemini response preview:
 1. The interface specification document shall define all necessary protocols, port numbers, logical addresses, data formats, and communication sequences required for software development and verification against interface requirements.
2. The system shall allow an authorized administrator to add a new network member within 5 minutes, requiring no more than 3 distinct user interface interactions, a
Added 8 rows for 2003 - tachonet.txt


Processing files: 100%|██████████| 79/79 [30:40<00:00, 23.30s/it]

File-wise detection & rewriting complete!
Results saved to: /content/drive/MyDrive/NLP_ASG/Outputs/Vague_Rewrites.csv


In [ ]:
from itertools import chain

# Load generated output
CSV_PATH = "/content/drive/MyDrive/NLP_ASG/Outputs/Vague_Rewrites.csv"
df = pd.read_csv(CSV_PATH)

terms_split = df["vague_term(s)"].dropna().apply(lambda x: [t.strip().lower() for t in x.split(",")])

# Flatten list of lists → single list of all vague terms
all_terms = list(chain.from_iterable(terms_split))

# Count frequency of each unique vague term
term_freq = pd.Series(all_terms).value_counts().reset_index()
term_freq.columns = ["Type of Bad Smell(Vague Term)", "Frequency"]

# printing first 15 rows of output
print(term_freq.head(15))

Summary_CSV = "/content/drive/MyDrive/NLP_ASG/Outputs/Summary.csv"
term_freq.to_csv(Summary_CSV, index=False, encoding="utf-8")

print(f"Per-term frequency table saved to {Summary_CSV}")